In [0]:
%python
df_raw = (
    spark.read
         .option("recursiveFileLookup", "true")
         .option("multiLine", "true")
         .json("/Volumes/workspace/default/json_raw_volume/IOT Device Status Messages/")
)

df_raw.printSchema() 

root
 |-- Body: struct (nullable = true)
 |    |-- code: string (nullable = true)
 |    |-- mnsn: string (nullable = true)
 |    |-- scheduleName: string (nullable = true)
 |    |-- smsProperties: struct (nullable = true)
 |    |    |-- device: struct (nullable = true)
 |    |    |    |-- address: string (nullable = true)
 |    |    |    |-- disposalMarkerList: array (nullable = true)
 |    |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |    |-- amount: struct (nullable = true)
 |    |    |    |    |    |    |-- capacity: string (nullable = true)
 |    |    |    |    |    |    |-- state: string (nullable = true)
 |    |    |    |    |    |    |-- typical: string (nullable = true)
 |    |    |    |    |    |    |-- unit: string (nullable = true)
 |    |    |    |    |    |-- color: string (nullable = true)
 |    |    |    |    |    |-- description: string (nullable = true)
 |    |    |    |    |    |-- type: string (nullable = true)
 |    |    |    |-- fam

In [0]:
from pyspark.sql.functions import col, lit, struct

df_device_latest = df_raw.select(

    # -------------------------
    # Identifiers
    # -------------------------
    col("SystemProperties.connectionDeviceId").alias("_id"),
    col("Body.timestamp").cast("long").alias("createTimestamp"),
    col("Body.type").alias("type"),

    # -------------------------
    # Fixed values
    # -------------------------
    lit("assigned").alias("attribute"),
    col("Properties.relatedGroupId").alias("relatedGroupId"),
    lit(None).cast("array<string>").alias("tagIds"),

    # -------------------------
    # Customer / dealer
    # -------------------------
    lit(None).cast("string").alias("communicationStatus"),
    col("Properties.customerId").alias("customerId"),
    col("Properties.dealerId").alias("dealerId"),
    lit(None).cast("string").alias("L1DealerId"),
    lit(None).cast("string").alias("L2DealerId"),
    lit(None).cast("string").alias("L3DealerId"),
    lit(None).cast("string").alias("divisionId"),

    # -------------------------
    # Commercial
    # -------------------------
    lit(None).cast("boolean").alias("onContract"),
    lit(None).cast("long").alias("purchaseDate"),
    lit(None).cast("string").alias("equipmentId"),
    lit(None).cast("boolean").alias("isSNAP"),
    lit(None).cast("string").alias("servicingDealerId"),
    lit(None).cast("string").alias("SAP_name"),
    lit(None).cast("string").alias("SAP_location"),

    # -------------------------
    # deviceGeneral
    # -------------------------
    struct(
        lit(None).cast("string").alias("relatedAgentId"),
        col("Body.smsProperties.device.modelName").alias("modelName"),
        col("Body.smsProperties.device.serialId").alias("serialNumber"),
        col("Body.smsProperties.device.familyName").alias("familyName"),
        col("Body.smsProperties.device.friendlyName").alias("friendlyName"),
        lit(None).cast("int").alias("dispErrorCode")
    ).alias("deviceGeneral"),

    # -------------------------
    # interface (INSIDE Body)
    # -------------------------
    struct(
        col("Body.smsProperties.interface.ethernetList").alias("ethernetList"),
        col("Body.smsProperties.interface.ipList").alias("ipList")
    ).alias("interface"),

    # -------------------------
    # Lists
    # -------------------------
    col("Body.smsProperties.device.inTrayList").alias("inTrayList"),
    col("Body.smsProperties.device.outTrayList").alias("outTrayList"),
    col("Body.smsProperties.device.disposalMarkerList").alias("disposalMarkerList"),
    col("Body.smsProperties.device.markerList").alias("markerList"),

    # -------------------------
    # Not present → NULL but with proper types
    # -------------------------
    lit(None).cast("string").alias("printerDevice"),
    lit(None).cast("array<struct<id:string,value:string>>").alias("maintenanceCodeList"),
    lit(None).cast("array<struct<id:string,value:string>>").alias("serviceCodeList")

)


In [0]:
df_device_latest.printSchema()

root
 |-- _id: string (nullable = true)
 |-- createTimestamp: long (nullable = true)
 |-- type: string (nullable = true)
 |-- attribute: string (nullable = false)
 |-- relatedGroupId: string (nullable = true)
 |-- tagIds: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- communicationStatus: string (nullable = true)
 |-- customerId: string (nullable = true)
 |-- dealerId: string (nullable = true)
 |-- L1DealerId: string (nullable = true)
 |-- L2DealerId: string (nullable = true)
 |-- L3DealerId: string (nullable = true)
 |-- divisionId: string (nullable = true)
 |-- onContract: boolean (nullable = true)
 |-- purchaseDate: long (nullable = true)
 |-- equipmentId: string (nullable = true)
 |-- isSNAP: boolean (nullable = true)
 |-- servicingDealerId: string (nullable = true)
 |-- SAP_name: string (nullable = true)
 |-- SAP_location: string (nullable = true)
 |-- deviceGeneral: struct (nullable = false)
 |    |-- relatedAgentId: string (nullable = true)
 |    |-

In [0]:
df_device_latest.display()

_id,createTimestamp,type,attribute,relatedGroupId,tagIds,communicationStatus,customerId,dealerId,L1DealerId,L2DealerId,L3DealerId,divisionId,onContract,purchaseDate,equipmentId,isSNAP,servicingDealerId,SAP_name,SAP_location,deviceGeneral,interface,inTrayList,outTrayList,disposalMarkerList,markerList,printerDevice,maintenanceCodeList,serviceCodeList
26e40cb8-53cc-4ddf-829e-7a0662fdcc5c,1765690085229,smsProps,assigned,6de069dd-89d2-43b5-bc64-15ffbba5ccd6,null,null,b8bf935d-026c-426a-b9f2-ef97d01bf973,281083,null,null,null,null,null,null,null,null,null,null,null,"List(null, SHARP BP-50C26, 2512055000, SHARP-155, , null)","List(List(List(14-50-51-CA-30-99, 1, 6), List(, 3, 24)), List(List(127.0.0.1, 172.29.240.22, 3, 255.0.0.0), List(172.29.241.167, 172.29.240.22, 1, 255.255.254.0)))","List(List(List(100, empty, 0, sheets), Bypass Tray, 1, false, true, -2, -2, unknown, other, -2, unknown, micrometers, -2, custom-media-type-stationery-stationery-1, Bypass Tray, Bypass Tray, false), List(List(550, , 181, sheets), Tray 1, 2, false, false, 210000, 297000, iso-a4-white, 210000 x 297000, 297000, A4, micrometers, 210000, custom-media-type-stationery-stationery-1, TRAY 1, Tray 1, false), List(List(550, empty, 0, sheets), Tray 2, 3, false, false, 420000, 297000, iso-a3-white, 420000 x 297000, 297000, A3R, micrometers, 420000, custom-media-type-stationery-stationery-1, BP-DE12, Tray 2, false), List(List(, unknown, , sheets), Auto Select, 31, false, false, -2, -2, unknown, other, -2, unknown, micrometers, -2, unknown, Auto Select, Auto Select, false))","List(List(List(400, notFull, , sheets), faceDown, Center Tray, 1, Center Tray, Center Tray, firstToLast))","List(List(List(, dis_ok, 100, percent), unknown, Waste Toner, wasteToner))","List(List(List(100, ok, 14, percent), cyan, Cyan Toner, toner), List(List(100, ok, 26, percent), magenta, Magenta Toner, toner), List(List(100, ok, 94, percent), yellow, Yellow Toner, toner), List(List(100, ok, 5, percent), black, Black Toner, toner), List(List(100, ok, 95, percent), cyan, Cyan Photoconductive Drum (DC), opc), List(List(100, ok, 95, percent), magenta, Magenta Photoconductive Drum (DM), opc), List(List(100, ok, 95, percent), yellow, Yellow Photoconductive Drum (DY), opc), List(List(100, ok, 93, percent), black, Black Photoconductive Drum (DK), opc))",null,null,null
26e40cb8-53cc-4ddf-829e-7a0662fdcc5c,1765690085343,smsProps,assigned,6de069dd-89d2-43b5-bc64-15ffbba5ccd6,null,null,b8bf935d-026c-426a-b9f2-ef97d01bf973,281083,null,null,null,null,null,null,null,null,null,null,null,"List(null, SHARP BP-50C26, 2512055000, SHARP-155, , null)","List(List(List(14-50-51-CA-30-99, 1, 6), List(, 3, 24)), List(List(127.0.0.1, 172.29.240.22, 3, 255.0.0.0), List(172.29.241.167, 172.29.240.22, 1, 255.255.254.0)))",null,null,null,null,null,null,null
26e40cb8-53cc-4ddf-829e-7a0662fdcc5c,1765690085763,smsProps,assigned,6de069dd-89d2-43b5-bc64-15ffbba5ccd6,null,null,b8bf935d-026c-426a-b9f2-ef97d01bf973,281083,null,null,null,null,null,null,null,null,null,null,null,"List(null, SHARP MX-3560N, 7506932600, SHARP-112, , null)","List(List(List(80-38-96-69-12-A9, 1, 6), List(, 2, 24)), List(List(127.0.0.1, 172.29.242.129, 2, 255.0.0.0), List(172.29.242.156, 172.29.242.129, 1, 255.255.255.128)))","List(List(List(100, empty, 0, sheets), Bypass Tray, 1, false, true, -2, -2, unknown, other, -2, unknown, micrometers, -2, stationery, Bypass Tray, Bypass Tray, false), List(List(600, , 198, sheets), Tray 1, 2, false, false, 210000, 297000, iso-a4-white, 210000 x 297000, 297000, A4, micrometers, 210000, stationery, TRAY 1, Tray 1, false), List(List(, unknown, , sheets), Auto Select, 31, false, false, -2, -2, unknown, other, -2, unknown, micrometers, -2, unknown, , Auto Select, true))","List(List(List(250, notFull, , sheets), faceDown, Center Tray, 1, Center Tray, Center Tray, firstToLast), List(List(100, notFull, , sheets), faceDown, Upper Tray, 3, MX-TR20, Upper Tray, firstToLast))","List(List(List(, dis_ok, 100, percent), unknown

In [0]:
df_task3 = df_device_latest.dropDuplicates(
    ["_id", "createTimestamp"]
)


In [0]:
df_task3.display()

_id,createTimestamp,type,attribute,relatedGroupId,tagIds,communicationStatus,customerId,dealerId,L1DealerId,L2DealerId,L3DealerId,divisionId,onContract,purchaseDate,equipmentId,isSNAP,servicingDealerId,SAP_name,SAP_location,deviceGeneral,interface,inTrayList,outTrayList,disposalMarkerList,markerList,printerDevice,maintenanceCodeList,serviceCodeList
26e40cb8-53cc-4ddf-829e-7a0662fdcc5c,1765679285143,smsProps,assigned,6de069dd-89d2-43b5-bc64-15ffbba5ccd6,null,null,b8bf935d-026c-426a-b9f2-ef97d01bf973,281083,null,null,null,null,null,null,null,null,null,null,null,"List(null, SHARP BP-50C26, 2512055000, SHARP-155, , null)","List(List(List(14-50-51-CA-30-99, 1, 6), List(, 3, 24)), List(List(127.0.0.1, 172.29.240.22, 3, 255.0.0.0), List(172.29.241.167, 172.29.240.22, 1, 255.255.254.0)))","List(List(List(100, empty, 0, sheets), Bypass Tray, 1, false, true, -2, -2, unknown, other, -2, unknown, micrometers, -2, custom-media-type-stationery-stationery-1, Bypass Tray, Bypass Tray, false), List(List(550, , 181, sheets), Tray 1, 2, false, false, 210000, 297000, iso-a4-white, 210000 x 297000, 297000, A4, micrometers, 210000, custom-media-type-stationery-stationery-1, TRAY 1, Tray 1, false), List(List(550, empty, 0, sheets), Tray 2, 3, false, false, 420000, 297000, iso-a3-white, 420000 x 297000, 297000, A3R, micrometers, 420000, custom-media-type-stationery-stationery-1, BP-DE12, Tray 2, false), List(List(, unknown, , sheets), Auto Select, 31, false, false, -2, -2, unknown, other, -2, unknown, micrometers, -2, unknown, Auto Select, Auto Select, false))","List(List(List(400, notFull, , sheets), faceDown, Center Tray, 1, Center Tray, Center Tray, firstToLast))","List(List(List(, dis_ok, 100, percent), unknown, Waste Toner, wasteToner))","List(List(List(100, ok, 14, percent), cyan, Cyan Toner, toner), List(List(100, ok, 26, percent), magenta, Magenta Toner, toner), List(List(100, ok, 94, percent), yellow, Yellow Toner, toner), List(List(100, ok, 5, percent), black, Black Toner, toner), List(List(100, ok, 95, percent), cyan, Cyan Photoconductive Drum (DC), opc), List(List(100, ok, 95, percent), magenta, Magenta Photoconductive Drum (DM), opc), List(List(100, ok, 95, percent), yellow, Yellow Photoconductive Drum (DY), opc), List(List(100, ok, 93, percent), black, Black Photoconductive Drum (DK), opc))",null,null,null
26e40cb8-53cc-4ddf-829e-7a0662fdcc5c,1765679285318,smsProps,assigned,6de069dd-89d2-43b5-bc64-15ffbba5ccd6,null,null,b8bf935d-026c-426a-b9f2-ef97d01bf973,281083,null,null,null,null,null,null,null,null,null,null,null,"List(null, SHARP MX-3560N, 7506932600, SHARP-112, , null)","List(List(List(80-38-96-69-12-A9, 1, 6), List(, 2, 24)), List(List(127.0.0.1, 172.29.242.129, 2, 255.0.0.0), List(172.29.242.156, 172.29.242.129, 1, 255.255.255.128)))","List(List(List(100, empty, 0, sheets), Bypass Tray, 1, false, true, -2, -2, unknown, other, -2, unknown, micrometers, -2, stationery, Bypass Tray, Bypass Tray, false), List(List(600, , 198, sheets), Tray 1, 2, false, false, 210000, 297000, iso-a4-white, 210000 x 297000, 297000, A4, micrometers, 210000, stationery, TRAY 1, Tray 1, false), List(List(, unknown, , sheets), Auto Select, 31, false, false, -2, -2, unknown, other, -2, unknown, micrometers, -2, unknown, , Auto Select, true))","List(List(List(250, notFull, , sheets), faceDown, Center Tray, 1, Center Tray, Center Tray, firstToLast), List(List(100, notFull, , sheets), faceDown, Upper Tray, 3, MX-TR20, Upper Tray, firstToLast))","List(List(List(, dis_ok, 100, percent), unknown, Waste Toner, wasteToner))","List(List(List(100, ok, 88, percent), cyan, Cyan Toner, toner), List(List(100, ok, 28, percent), magenta, Magenta Toner, toner), List(List(100, ok, 19, percent), yellow, Yellow Toner, toner), List(List(100, ok, 100, percent), black, Black Toner, toner), List(List(100, ok, 50, percent), cyan, Cyan Photoconductive Drum, opc), List(List(100, ok, 50, percent), magenta, Magenta Photoconductive Drum, opc), List(List(100, ok, 50, percent), yellow, Y

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, desc, col

window_spec = Window.partitionBy("_id").orderBy(desc("createTimestamp"))

df_latest = (
    df_task3
    .withColumn("rn", row_number().over(window_spec))
    .filter(col("rn") == 1)
    .drop("rn")
)


In [0]:
df_latest.display()

_id,createTimestamp,type,attribute,relatedGroupId,tagIds,communicationStatus,customerId,dealerId,L1DealerId,L2DealerId,L3DealerId,divisionId,onContract,purchaseDate,equipmentId,isSNAP,servicingDealerId,SAP_name,SAP_location,deviceGeneral,interface,inTrayList,outTrayList,disposalMarkerList,markerList,printerDevice,maintenanceCodeList,serviceCodeList
26e40cb8-53cc-4ddf-829e-7a0662fdcc5c,1765765686727,smsProps,assigned,6de069dd-89d2-43b5-bc64-15ffbba5ccd6,null,null,b8bf935d-026c-426a-b9f2-ef97d01bf973,281083,null,null,null,null,null,null,null,null,null,null,null,"List(null, SHARP BP-50C26, 3333333323, SHARP-155, , null)","List(List(List(16-47-49-16-34-93, 1, 6), List(, 3, 24)), List(List(127.0.0.1, 172.29.240.22, 3, 255.0.0.0), List(172.29.241.167, 172.29.240.22, 1, 255.255.254.0)))","List(List(List(57, empty, 0, sheets), Bypass Tray, 1, false, true, -2, -2, unknown, other, -2, unknown, micrometers, -2, custom-media-type-stationery-stationery-1, Bypass Tray, Bypass Tray, false), List(List(666, , 181, sheets), Tray 1, 2, false, false, 390000, 270000, iso_sra4_225x320mm-white, 390000 x 270000, 270000, 8KR, micrometers, 390000, custom-media-type-stationery-stationery-1, TRAY 1, Tray 1, false), List(List(666, empty, 0, sheets), Tray 2, 3, false, false, 215900, 279400, na-letter-white, 215900 x 279400, 279400, LETTER, micrometers, 215900, custom-media-type-stationery-stationery-1, BP-DE12, Tray 2, false), List(List(, unknown, , sheets), Auto Select, 31, false, false, -2, -2, unknown, other, -2, unknown, micrometers, -2, unknown, Auto Select, Auto Select, false))","List(List(List(400, full, 0, sheets), faceDown, Center Tray, 1, Center Tray, Center Tray, firstToLast))","List(List(List(, unknown, , percent), unknown, Waste Toner, wasteToner))","List(List(List(100, low, , percent), cyan, Cyan Toner, toner), List(List(100, ok, 92, percent), magenta, Magenta Toner, toner), List(List(100, ok, 25, percent), yellow, Yellow Toner, toner), List(List(100, ok, 55, percent), black, Black Toner, toner), List(List(100, ok, 99, percent), cyan, Cyan Photoconductive Drum (DC), opc), List(List(100, ok, 99, percent), magenta, Magenta Photoconductive Drum (DM), opc), List(List(100, ok, 99, percent), yellow, Yellow Photoconductive Drum (DY), opc), List(List(100, ok, 99, percent), black, Black Photoconductive Drum (DK), opc))",null,null,null
89a2dbb3-09af-420e-8086-d9da584a8de5,1765746156156,offline,assigned,2344cb97-4ebe-4412-8a8c-64730003681c,null,null,ea387dfe-357c-48b7-818a-e01063c184f5,281083,null,null,null,null,null,null,null,null,null,null,null,"List(null, null, null, null, null, null)","List(null, null)",null,null,null,null,null,null,null


In [0]:
spark.sql("DROP TABLE IF EXISTS device_latest")


DataFrame[]

In [0]:
df_latest.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable("device_latest")


In [0]:
from delta.tables import DeltaTable

delta_table = DeltaTable.forName(spark, "device_latest")

(
    delta_table.alias("t")
    .merge(
        df_latest.alias("s"),
        "t._id = s._id"
    )
    .whenMatchedUpdate(
        condition="s.createTimestamp > t.createTimestamp",
        set={
            "_id": "s._id",
            "createTimestamp": "s.createTimestamp",
            "type": "s.type",
            "attribute": "s.attribute",
            "relatedGroupId": "s.relatedGroupId",
            "tagIds": "s.tagIds",
            "communicationStatus": "s.communicationStatus",
            "customerId": "s.customerId",
            "dealerId": "s.dealerId",
            "L1DealerId": "s.L1DealerId",
            "L2DealerId": "s.L2DealerId",
            "L3DealerId": "s.L3DealerId",
            "divisionId": "s.divisionId",
            "onContract": "s.onContract",
            "purchaseDate": "s.purchaseDate",
            "equipmentId": "s.equipmentId",
            "isSNAP": "s.isSNAP",
            "servicingDealerId": "s.servicingDealerId",
            "SAP_name": "s.SAP_name",
            "SAP_location": "s.SAP_location",
            "deviceGeneral": "s.deviceGeneral",
            "interface": "s.interface",
            "inTrayList": "s.inTrayList",
            "outTrayList": "s.outTrayList",
            "disposalMarkerList": "s.disposalMarkerList",
            "markerList": "s.markerList",
            "printerDevice": "s.printerDevice",
            "maintenanceCodeList": "s.maintenanceCodeList",
            "serviceCodeList": "s.serviceCodeList"
        }
    )
    .whenNotMatchedInsertAll()
    .execute()
)


DataFrame[num_affected_rows: bigint, num_updated_rows: bigint, num_deleted_rows: bigint, num_inserted_rows: bigint]

In [0]:
%sql
SELECT _id, createTimestamp
FROM device_latest
ORDER BY _id;


_id,createTimestamp
26e40cb8-53cc-4ddf-829e-7a0662fdcc5c,1765765686727
89a2dbb3-09af-420e-8086-d9da584a8de5,1765746156156


In [0]:
%sql
SELECT _id, COUNT(*) 
FROM device_latest
GROUP BY _id
HAVING COUNT(*) > 1;


_id,COUNT(*)
